## This notebook combines all the raw dirty datasets from Data-dirty folder, drops extra columns from it, remove duplicates and NaN rows, add useful columns,etc. and then saves the cleaned dataset as one csv file.

In [1]:
import pandas as pd
import numpy as np
import os
import string
import nltk
from nltk.corpus import stopwords

## 1. First, we combine all dirty datasets from the files for all months and years in the Dirty-data folder

In [2]:
path = './Data-new/Data-dirty'
files = os.listdir(path)

In [3]:
# Append dirty data from all available months and years
df = pd.DataFrame()
for name in files:
    tmp = pd.read_csv(os.path.join(path, name))
    df = df.append(tmp)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/pandas/core/frame.py:7138: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort,


In [4]:
len(df)

58489

In [5]:
df.head()

,listing_value,remove,tweets_Date,tweets_Images,tweets_Likes,tweets_Name,tweets_Replies,tweets_Retweets,tweets_Text,tweets_hrefURL,tweets_imageURL
0,1.0,1,"Dec 31, 2012",NaN,81,CNN Breaking News,100,983,Doctors: Secretary of State Hillary #Clinton h...,NaN,NaN
1,1.0,1,"Dec 31, 2012",NaN,39,CNN Breaking News,7,239,"North Carolina governor pardons ""Wilmington 10...",NaN,NaN
2,1.0,1,"Dec 31, 2012",NaN,13,CNN Breaking News,73,412,"No House vote today on fiscal cliff, House lea...",NaN,NaN
3,1.0,1,"Dec 31, 2012",NaN,33,CNN Breaking News,21,213,Senate Minority Leader Mitch McConnell says fi...,NaN,NaN
4,1.0,1,"Dec 31, 2012",NaN,39,CNN Breaking News,29,418,"Nine dead, 18 injured after bus skids on ice a...",NaN,NaN


## 2. Now we drop duplicate rows, drop rows with NaN for tweets_Date. We create two extra binary columns, and then drop all other redundant columns. 

In [6]:
# First drop duplicate rows:
df.drop_duplicates()


# Next, drop rows which have tweets_Date == NaN
df.dropna(subset=['tweets_Date'], inplace=True) 
df.dropna(subset=['tweets_Text'], inplace=True) 


# Then drop irrelevant columns as follow:

#df = df.drop(labels=['listing_value', 'tweets_Name', 'remove'], axis=1).reset_index(drop=True)


# Now Add a binary column showing whether a tweet has image and/or external links
df['has_image'] = (pd.notnull(df['tweets_Images'])).astype(int)
df['has_Link'] = (pd.notnull(df['tweets_hrefURL'])).astype(int)

# Next, drop columns tweets_Images, tweets_imageURL, tweets_hrefURL, Unnamed
df = df.drop(labels=['tweets_Images', 'tweets_imageURL', 'tweets_hrefURL'], axis=1).reset_index(drop=True)


In [7]:
df

,listing_value,remove,tweets_Date,tweets_Likes,tweets_Name,tweets_Replies,tweets_Retweets,tweets_Text,has_image,has_Link
0,1.0,1,"Dec 31, 2012",81,CNN Breaking News,100,983,Doctors: Secretary of State Hillary #Clinton h...,0,0
1,1.0,1,"Dec 31, 2012",39,CNN Breaking News,7,239,"North Carolina governor pardons ""Wilmington 10...",0,0
2,1.0,1,"Dec 31, 2012",13,CNN Breaking News,73,412,"No House vote today on fiscal cliff, House lea...",0,0
3,1.0,1,"Dec 31, 2012",33,CNN Breaking News,21,213,Senate Minority Leader Mitch McConnell says fi...,0,0
4,1.0,1,"Dec 31, 2012",39,CNN Breaking News,29,418,"Nine dead, 18 injured after bus skids on ice a...",0,0
...,...,...,...,...,...,...,...,...,...,...
38174,NaN,1,"Nov 17, 2015",208,CNN Breaking News,23,158,"Before Paris attacks, France and allies tried ...",0,0
38175,NaN,1,"Nov 17, 2015",519,CNN Breaking News,108,409,Porsche blames actor Paul Walker for his own d...,0,0
38176,NaN,1,"Nov 17, 2015",526,CNN Breaking News,56,935,Russia says the passenger jet that crashed ove...,0,0
38177,NaN,1,"Nov 17, 2015",336,CNN Breaking News,27,498,Yakuza boss of Japan's largest gang syndicate ...,1,1


## 3. We standardize the number of likes, etc. Then we also clean up tweet texts.

In [8]:
# Method to standardize numbers ==> example: 4k should be 4000
def standardized_replies_retweets_likes(df):
    for i in range(len(df)):
        if 'K' in str(df.at[i,'tweets_Replies']):
            temp = df.at[i,'tweets_Replies'].replace('K','')
            df.at[i,'tweets_Replies'] = float(temp)*1000
        if 'K' in str(df.at[i,'tweets_Retweets']):
            temp = df.at[i,'tweets_Retweets'].replace('K','')
            df.at[i,'tweets_Retweets'] = float(temp)*1000
        if 'K' in str(df.at[i,'tweets_Likes']):
            temp = df.at[i,'tweets_Likes'].replace('K','')
            df.at[i,'tweets_Likes'] = float(temp)*1000

        df.at[i,'tweets_Replies'] = float(df.at[i,'tweets_Replies'])
        df.at[i,'tweets_Retweets'] = float(df.at[i,'tweets_Retweets'])
        df.at[i,'tweets_Likes'] = float(df.at[i,'tweets_Likes'])
            
    return df

In [9]:
df = standardized_replies_retweets_likes(df)

In [10]:
df.head()

,listing_value,remove,tweets_Date,tweets_Likes,tweets_Name,tweets_Replies,tweets_Retweets,tweets_Text,has_image,has_Link
0,1.0,1,"Dec 31, 2012",81,CNN Breaking News,100,983,Doctors: Secretary of State Hillary #Clinton h...,0,0
1,1.0,1,"Dec 31, 2012",39,CNN Breaking News,7,239,"North Carolina governor pardons ""Wilmington 10...",0,0
2,1.0,1,"Dec 31, 2012",13,CNN Breaking News,73,412,"No House vote today on fiscal cliff, House lea...",0,0
3,1.0,1,"Dec 31, 2012",33,CNN Breaking News,21,213,Senate Minority Leader Mitch McConnell says fi...,0,0
4,1.0,1,"Dec 31, 2012",39,CNN Breaking News,29,418,"Nine dead, 18 injured after bus skids on ice a...",0,0


In [11]:
def text_cleaning(data):
    #lower case
    data['tweets_Text']=data['tweets_Text'].str.lower()
    
    #remove punctuation
    punc_to_remove = string.punctuation
    def remove_punctuation(text):
        return text.translate(str.maketrans('', '', punc_to_remove))
    data['tweets_Text'] = data['tweets_Text'].apply(lambda text: remove_punctuation(text))
    
    #remove stop words
    ", ".join(stopwords.words('english'))
    STOPWORDS = set(stopwords.words('english'))
    def remove_stopwords(text):
        return " ".join([word for word in str(text).split() if word not in STOPWORDS])

    data['tweets_Text'] = data['tweets_Text'].apply(lambda text: remove_stopwords(text))

    return data


    

In [12]:
df = text_cleaning(df)

In [13]:
df.head()

,listing_value,remove,tweets_Date,tweets_Likes,tweets_Name,tweets_Replies,tweets_Retweets,tweets_Text,has_image,has_Link
0,1.0,1,"Dec 31, 2012",81,CNN Breaking News,100,983,doctors secretary state hillary clinton blood ...,0,0
1,1.0,1,"Dec 31, 2012",39,CNN Breaking News,7,239,north carolina governor pardons wilmington 10 ...,0,0
2,1.0,1,"Dec 31, 2012",13,CNN Breaking News,73,412,house vote today fiscal cliff house leaders te...,0,0
3,1.0,1,"Dec 31, 2012",33,CNN Breaking News,21,213,senate minority leader mitch mcconnell says fi...,0,0
4,1.0,1,"Dec 31, 2012",39,CNN Breaking News,29,418,nine dead 18 injured bus skids ice tumbles emb...,0,0


In [14]:
df.dropna(subset=['tweets_Text'], inplace=True) 

In [15]:
df

,listing_value,remove,tweets_Date,tweets_Likes,tweets_Name,tweets_Replies,tweets_Retweets,tweets_Text,has_image,has_Link
0,1.0,1,"Dec 31, 2012",81,CNN Breaking News,100,983,doctors secretary state hillary clinton blood ...,0,0
1,1.0,1,"Dec 31, 2012",39,CNN Breaking News,7,239,north carolina governor pardons wilmington 10 ...,0,0
2,1.0,1,"Dec 31, 2012",13,CNN Breaking News,73,412,house vote today fiscal cliff house leaders te...,0,0
3,1.0,1,"Dec 31, 2012",33,CNN Breaking News,21,213,senate minority leader mitch mcconnell says fi...,0,0
4,1.0,1,"Dec 31, 2012",39,CNN Breaking News,29,418,nine dead 18 injured bus skids ice tumbles emb...,0,0
...,...,...,...,...,...,...,...,...,...,...
38174,NaN,1,"Nov 17, 2015",208,CNN Breaking News,23,158,paris attacks france allies tried target isis ...,0,0
38175,NaN,1,"Nov 17, 2015",519,CNN Breaking News,108,409,porsche blames actor paul walker death one spo...,0,0
38176,NaN,1,"Nov 17, 2015",526,CNN Breaking News,56,935,russia says passenger jet crashed egypt octobe...,0,0
38177,NaN,1,"Nov 17, 2015",336,CNN Breaking News,27,498,yakuza boss japans largest gang syndicate foun...,1,1


In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 38179 entries, 0 to 38178
Data columns (total 10 columns):
listing_value      27475 non-null float64
remove             38179 non-null int64
tweets_Date        38179 non-null object
tweets_Likes       38175 non-null object
tweets_Name        38179 non-null object
tweets_Replies     35322 non-null object
tweets_Retweets    38179 non-null object
tweets_Text        38179 non-null object
has_image          38179 non-null int64
has_Link           38179 non-null int64
dtypes: float64(1), int64(3), object(6)
memory usage: 3.2+ MB


# Change NaN replies to 0

In [17]:
values = {'tweets_Replies': 0}
df.fillna(value=values, inplace=True)

In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 38179 entries, 0 to 38178
Data columns (total 10 columns):
listing_value      27475 non-null float64
remove             38179 non-null int64
tweets_Date        38179 non-null object
tweets_Likes       38175 non-null object
tweets_Name        38179 non-null object
tweets_Replies     38179 non-null float64
tweets_Retweets    38179 non-null object
tweets_Text        38179 non-null object
has_image          38179 non-null int64
has_Link           38179 non-null int64
dtypes: float64(2), int64(3), object(5)
memory usage: 3.2+ MB


## 4. Now categorize the values of tweets_Replies column and add a new column which presents the categorical variable: ADD CODE HERE FOR CATEGORIES

In [19]:
df['tweets_ResponseCategory'], bins = pd.qcut(df['tweets_Replies'], 3, labels=[0, 1, 2], retbins = True)


In [20]:
df.groupby('tweets_ResponseCategory').size()

tweets_ResponseCategory
0    12816
1    12858
2    12505
dtype: int64

## 5. Remove the NaNs and index columns

In [21]:
df.dropna(subset=['tweets_Text'], inplace=True)
df = df.loc[:, ~df.columns.str.contains('^Unnamed')]

df.head()

,listing_value,remove,tweets_Date,tweets_Likes,tweets_Name,tweets_Replies,tweets_Retweets,tweets_Text,has_image,has_Link,tweets_ResponseCategory
0,1.0,1,"Dec 31, 2012",81,CNN Breaking News,100.0,983,doctors secretary state hillary clinton blood ...,0,0,2
1,1.0,1,"Dec 31, 2012",39,CNN Breaking News,7.0,239,north carolina governor pardons wilmington 10 ...,0,0,0
2,1.0,1,"Dec 31, 2012",13,CNN Breaking News,73.0,412,house vote today fiscal cliff house leaders te...,0,0,2
3,1.0,1,"Dec 31, 2012",33,CNN Breaking News,21.0,213,senate minority leader mitch mcconnell says fi...,0,0,1
4,1.0,1,"Dec 31, 2012",39,CNN Breaking News,29.0,418,nine dead 18 injured bus skids ice tumbles emb...,0,0,1


## 5. Drop tweets_Date, tweets_Replies, tweets_Retweets, and tweets_Likes and save the final dataset as a csv file. Please indicate in the name of csv weather it has been preprocessed using this notebook.
## We can save two versions, one with the above columns dropped and one including these columns just in case. Please mention in the name which is version is what along with time range.

In [22]:
df.to_csv('./Data-new/Data-clean/Preprocessed_2011-2017mid_all_cols.csv', index = False)

In [23]:
df.drop(['tweets_Date', 'tweets_Likes', 'tweets_Replies', 'tweets_Retweets','listing_value', 'remove', 'tweets_Name'], axis=1, inplace=True)

In [24]:
df.head()

,tweets_Text,has_image,has_Link,tweets_ResponseCategory
0,doctors secretary state hillary clinton blood ...,0,0,2
1,north carolina governor pardons wilmington 10 ...,0,0,0
2,house vote today fiscal cliff house leaders te...,0,0,2
3,senate minority leader mitch mcconnell says fi...,0,0,1
4,nine dead 18 injured bus skids ice tumbles emb...,0,0,1


In [25]:
df.to_csv('./Data-new/Data-clean/Preprocessed_2011-2017mid_short.csv', index = False)

In [26]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 38179 entries, 0 to 38178
Data columns (total 4 columns):
tweets_Text                38179 non-null object
has_image                  38179 non-null int64
has_Link                   38179 non-null int64
tweets_ResponseCategory    38179 non-null category
dtypes: category(1), int64(2), object(1)
memory usage: 1.2+ MB
